In [1]:
import pandas as pd
import pycountry
from tqdm.notebook import tqdm

In [2]:
# download old+new food balances and land use dataset (all data!) from FAOSTAT, rename them accordingly and place them in the directory of this notebook!
new = pd.read_csv('balances_new.csv', encoding='latin1')
new = new.loc[new['Area Code'] < 5000]
old = pd.read_csv('balances_old.csv', encoding='latin1')
old = old.loc[old['Area Code'] < 5000]
areas = pd.read_csv('areas.csv', encoding='latin1')
areas = areas.loc[(areas['Area Code'] < 5000) & (areas['Item Code'] == 6610) & (areas['Element Code'] == 5110)]

# also download item group and element definitions for balances dataset
item_groups = pd.read_csv('item_group_definitions.csv', encoding='latin1')
elements = pd.read_csv('element_definitions.csv', encoding='latin1')

In [3]:
# combine datasets
df = pd.merge(old, new, 'outer')
areas = areas.loc[areas['Area Code'].isin(df['Area Code'].unique())]
df = pd.merge(df, areas, 'outer')

# remove unit (e.g. 'kcal/capita/day') from food supply element labels
element_codes_to_fix = [684, 664, 645, 674]
df.loc[df['Element Code'] == 684, 'Element'] = "Fat supply quantity"
df.loc[df['Element Code'] == 664, 'Element'] = "Food supply"
df.loc[df['Element Code'] == 645, ['Element', 'Unit']] = ["Food supply quantity", "kg/capita/yr"]
df.loc[df['Element Code'] == 674, 'Element'] = "Protein supply quantity"

In [4]:
# merge duplicate area codes for north macedonia (154) and swaziland (209)
macedonia = df.loc[df['Area Code'] == 154].groupby(['Area Code', 'Item Code', 'Item', 'Element Code', 'Element', 'Unit']).sum().reset_index()
macedonia['Area'] = "North Macedonia"
swaziland = df.loc[df['Area Code'] == 209].groupby(['Area Code', 'Item Code', 'Item', 'Element Code', 'Element', 'Unit']).sum().reset_index()
swaziland['Area'] = "Eswatini"
# replace old records
df = df.loc[(df['Area Code'] != 154) & (df['Area Code'] != 209)]
df = df.append(macedonia, ignore_index=True, sort=False)
df = df.append(swaziland, ignore_index=True, sort=False)

# fix a few area names that cant be found otherwise
df.loc[df['Area Code'] == 236, 'Area'] = 'Venezuela'
df.loc[df['Area Code'] == 19, 'Area'] = 'Bolivia'
df.loc[df['Area Code'] == 102, 'Area'] = 'Iran'
df.loc[df['Area Code'] == 117, 'Area'] = 'South Korea'
df.loc[df['Area Code'] == 107, 'Area'] = 'Côte d\'Ivoire'

# change area codes to iso format
iso_codes = {}
for area_name in tqdm(df['Area'].unique()):
    country = pycountry.countries.get(name=area_name)
    if country:
        iso_codes[area_name] = country.alpha_3
    else:
        try: 
            iso_codes[area_name] = pycountry.countries.search_fuzzy(area_name)[0].alpha_3
        except LookupError:
            pass
    
tqdm.pandas()
df['Area Code'] = df['Area'].progress_apply(lambda area_name: iso_codes.get(area_name))

# remove entries of no unresolvable countries :(
df = df.drop(df.loc[df['Area Code'].isnull()].index)

/home/elias/anaconda3/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [5]:
# change units from thousands to single tonnes/persons
tqdm.pandas()
df['Unit'] = df['Unit'].progress_apply(lambda unit: unit[len("1000 "):] if unit.startswith("1000 ") else unit)
df.loc[(df['Unit'] == 'tonnes') | (df['Unit'] == 'persons') | (df['Unit'] == "ha"), 'Y1961':'Y2017'] *= 1000

In [6]:
year_cols = [col for col in df if col.startswith('Y')]
grouping_cols = ['Area Code', 'Area', 'Unit']

def calc_total(element_code, item_code):
    element = elements.loc[elements['Element Code'] == element_code, 'Element'].iloc[0]
    item_codes = item_groups.loc[item_groups['Item Group Code'] == item_code, 'Item Code']
    item = item_groups.loc[item_groups['Item Group Code'] == item_code, 'Item Group'].iloc[0]
    total = df.loc[(df['Element Code'] == element_code) & (df['Item Code'].isin(item_codes))].groupby(grouping_cols)[year_cols].sum()
    total['Item Code'], total['Item'], total['Element Code'], total['Element'] = [item_code, item, element_code, element]
    total = total.sort_values("Area Code").reset_index()
    return total

def calc_ratio(df1, df2, element, unit, scalar=1):
    element_code = df1['Element Code'].iloc[0]+1
    item_code = df1['Item Code'].iloc[0]
    item = df1['Item'].iloc[0]
    ratio = df1[year_cols] / df2[year_cols] * scalar
    ratio['Item Code'], ratio['Item'], ratio['Element Code'], ratio['Element'], ratio['Unit'] = [item_code, item, element_code, element, unit]
    ratio[['Area Code', 'Area']] = df1[['Area Code', 'Area']]
    return ratio
    
# calculate totals
total_production = calc_total(5511, 2901)
total_losses = calc_total(5123, 2901)
total_import = calc_total(5611, 2901)
total_export = calc_total(5911, 2901)
total_supply = calc_total(5301, 2901)
ap_production = calc_total(5511, 2941)
ap_losses = calc_total(5123, 2941)
ap_import = calc_total(5611, 2941)
ap_export = calc_total(5911, 2941)
ap_supply = calc_total(5301, 2941)
vp_production = calc_total(5511, 2903)
vp_losses = calc_total(5123, 2903)
vp_import = calc_total(5611, 2903)
vp_export = calc_total(5911, 2903)
vp_supply = calc_total(5301, 2903)
    
# calculate ratios
export_ratio = calc_ratio(total_export, total_production, "Production Export Ratio", "%", 100)
import_ratio = calc_ratio(total_import, total_supply, "Domestic Supply Import Ratio", "%", 100)
losses_ratio = calc_ratio(total_losses, total_production, "Production Loss Ratio", "%", 100)
hectares = df.loc[(df['Item Code'] == 6610) & (df['Element Code'] == 5110)].sort_values('Area Code').reset_index(drop=True)
production_ratio = calc_ratio(total_production, hectares, "Production per Hectare", "tonnes/ha")

# append
df = df.append(total_production, ignore_index=True, sort=False)
df = df.append(total_losses, ignore_index=True, sort=False)
df = df.append(total_import, ignore_index=True, sort=False)
df = df.append(total_export, ignore_index=True, sort=False)
df = df.append(total_supply, ignore_index=True, sort=False)
df = df.append(ap_production, ignore_index=True, sort=False)
df = df.append(ap_losses, ignore_index=True, sort=False)
df = df.append(ap_import, ignore_index=True, sort=False)
df = df.append(ap_export, ignore_index=True, sort=False)
df = df.append(ap_supply, ignore_index=True, sort=False)
df = df.append(vp_production, ignore_index=True, sort=False)
df = df.append(vp_losses, ignore_index=True, sort=False)
df = df.append(vp_import, ignore_index=True, sort=False)
df = df.append(vp_export, ignore_index=True, sort=False)
df = df.append(vp_supply, ignore_index=True, sort=False)
df = df.append(export_ratio, ignore_index=True, sort=False)
df = df.append(import_ratio, ignore_index=True, sort=False)
df = df.append(losses_ratio, ignore_index=True, sort=False)
df = df.append(production_ratio, ignore_index=True, sort=False)

In [7]:
# sort and write to file
df = df.sort_values(['Area', 'Item', 'Element'])
df.to_csv("balances.csv", index=False)